In [ ]:
import numpy as np
import os 
import sys
import numba 
import matplotlib.pyplot as plt
import iminuit 
import scipy
import pandas

from template_fit import *

In [ ]:
df = pandas.read_csv("./pulse_seperation_standalone.csv", sep="|")
df = df.loc[df['noise'] == 1]
df.head()

In [ ]:
ti = 0
dfi = df.loc[df['t2'] == ti]

# plt.plot(dfi['e2'], dfi['nfound'])
fig,ax = plt.subplots(figsize=(7,5))
plt.scatter(dfi['e2'], dfi['e1'], c=dfi['nfound'])
plt.xlabel("E2 [Pulse Height]")
plt.ylabel("E1 [Pulse Height]")
plt.title(f"Pulse Resolution for t2={round(ti,2)}")
plt.colorbar()
plt.show()

In [ ]:
# ti = 0
# dfi = df.loc[df['t2'] == ti]

for ti, dfi in df.groupby(['t2']):

    # plt.plot(dfi['e2'], dfi['nfound'])
    fig,ax = plt.subplots(figsize=(7,5))
    plt.scatter(dfi['e2'], dfi['e1'], c=dfi['nfound'])
    plt.xlabel("E2 [Pulse Height]")
    plt.ylabel("E1 [Pulse Height]")
    plt.title(f"Pulse Resolution for t2={round(ti,2)}")
    plt.colorbar()
    plt.savefig(f"./images/pulse_resolution/pulses_e1_vs_e2_by_t2_{ti}.png", bbox_inches='tight', facecolor='w')
    plt.show()
    # break

In [ ]:
for ti, dfi in df.groupby(['e1']):
    # plt.plot(dfi['e2'], dfi['nfound'])
    fig,ax = plt.subplots(figsize=(7,5))
    plt.scatter(dfi['e2'], dfi['t2'], c=dfi['nfound'])
    plt.xlabel("E2 [Pulse Height]")
    plt.ylabel("T2 [ns]")
    plt.title(f"Pulse Resolution for E1={round(ti,2)}")
    plt.colorbar()
    plt.savefig(f"./images/pulse_resolution/pulses_e2_vs_t2_by_e1_{ti}.png", bbox_inches='tight', facecolor='w')
    plt.show()
    # break

---

# Investigate the pulses where there are 3 fits

---

In [ ]:
df3 = df.loc[df['nfound'] == 3]
df3.head()

In [ ]:
template = LXeTemplate.load('lxe.template')


def plot_fit(row):
    fig,ax = plt.subplots(figsize=(15,5))
    plt.plot([row['t1'], row['t2']], [0,0], "o")

    times = [row['fit_t0'], row['fit_t1'], row['fit_t2']]

    plt.plot(times, np.ones_like(times),"o")
    return fig,ax 

def recreate_fit(row):
    times = [row['t1'], row['t2']]
    energies = [row['e1'], row['e2']]
    # print(times,energies)
    data = make_fake_data_from_template(template.template,times, energies , noisefloor=10)
    fit = TemplateFit(data, template.template, verbose=False, chi2limit=100, minimum_energy=25)
    fit.do_fit()
    # fit.save("")
    return fit.plot()

for i, row in df3.iterrows():
    plot_fit(row)
    plt.show()
    recreate_fit(row)
    plt.show()
    break

In [ ]:
recreate_fit({'e1':310, "e2":5000, "t1":100, "t2":115})